# Exploratory Data Analysis

In [1]:
!pip install wandb

Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [3]:
!pip install fastai

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for fastai from https://files.pythonhosted.org/packages/45/2b/58f86cfc507e8da396ef9876ecdc5e196ac9ba99b0c85612ebe0c4e25c1c/fastai-2.7.17-py3-none-any.whl.metadata
  Obtaining dependency information for fastdownload<2,>=0.0.5 from https://files.pythonhosted.org/packages/47/60/ed35253a05a70b63e4f52df1daa39a6a464a3e22b0bd060b77f63e2e2b6a/fastdownload-0.0.7-py3-none-any.whl.metadata
  Obtaining dependency information for fastcore<1.8,>=1.5.29 from https://files.pythonhosted.org/packages/25/40/fb8f383bd725aea40c5a7479904035018d541d6249e0b8f29d8e17eff691/fastcore-1.7.8-py3-none-any.whl.metadata
  Obtaining dependency information for fastprogress>=0.2.4 from https://files.pythonhosted.org/packages/a7/8f/213223fdee199c55db81e2d0c669f30e8285c5be2526c4ed924de39247da/fastprogress-1.0.3-py3-none-any.whl.metadata
  Obtaining dependency information for spacy<4 from https://files.pythonhos

DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [5]:
WANDB_PROJECT = "mlops-course-001"
ENTITY = None
BDD_CLASSES = {i:c for i,c in enumerate(['background', 'road', 'traffic light', 'traffic sign', 'person', 'vehicle', 'bicycle'])}
RAW_DATA_AT = 'bdd_simple_1k'
PROCESSED_DATA_AT = 'bdd_simple_1k_split'

In [1]:
from fastai.vision.all import *
import wandb

In [3]:
URL = 'https://storage.googleapis.com/wandb_course/bdd_simple_1k.zip'

In [4]:
path = Path(untar_data(URL, force_download=True))
(path/'images').ls()

(#1000) [Path('C:/Users/Semal/.fastai/data/bdd_simple_1k/images/0027eed2-09c90000.jpg'),Path('C:/Users/Semal/.fastai/data/bdd_simple_1k/images/0027eed2-09c90001.jpg'),Path('C:/Users/Semal/.fastai/data/bdd_simple_1k/images/00aad4a0-ee8135fe.jpg'),Path('C:/Users/Semal/.fastai/data/bdd_simple_1k/images/00d79c0a-23befe54.jpg'),Path('C:/Users/Semal/.fastai/data/bdd_simple_1k/images/00e69ee0-9656df95.jpg'),Path('C:/Users/Semal/.fastai/data/bdd_simple_1k/images/00e9be89-00000130.jpg'),Path('C:/Users/Semal/.fastai/data/bdd_simple_1k/images/00e9be89-00000175.jpg'),Path('C:/Users/Semal/.fastai/data/bdd_simple_1k/images/00e9be89-00001025.jpg'),Path('C:/Users/Semal/.fastai/data/bdd_simple_1k/images/00e9be89-00001030.jpg'),Path('C:/Users/Semal/.fastai/data/bdd_simple_1k/images/00e9be89-00001070.jpg')...]

In [5]:
def label_func(fname):
    return (fname.parent.parent/"labels")/f"{fname.stem}_mask.png"

def get_classes_per_image(mask_data, class_labels):
    unique = list(np.unique(mask_data))
    result_dict = {}
    for _class in class_labels.keys():
        result_dict[class_labels[_class]] = int(_class in unique)
    return result_dict

def _create_table(image_files, class_labels):
    "Create a table with the dataset"
    labels = [str(class_labels[_lab]) for _lab in list(class_labels)]
    table = wandb.Table(columns=["File_Name", "P1", "P2", "Images", "Dataset"] + labels)
    
    for i, image_file in progress_bar(enumerate(image_files), total=len(image_files)):
        image = Image.open(image_file)
        mask_data = np.array(Image.open(label_func(image_file)))
        class_in_image = get_classes_per_image(mask_data, class_labels)
        table.add_data(
            image_file.stem,
            image_file.stem.split('-')[0],
            image_file.stem.split('-')[1],
            wandb.Image(
                    image,
                    masks={
                        "predictions": {
                            "mask_data": mask_data,
                            "class_labels": class_labels,
                        }
                    }
            ),
            "bdd1k", 
            *[class_in_image[_lab] for _lab in labels]
        )
    
    return table

In [6]:
run = wandb.init(project=WANDB_PROJECT, entity=ENTITY, job_type="upload")

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: semaljohari80. Use `wandb login --relogin` to force relogin


In [7]:
artifact = wandb.Artifact(RAW_DATA_AT, type="raw_data")

In [8]:
artifact.add_file(path/"LICENSE.txt", name="LICENSE.txt")

ArtifactManifestEntry(path='LICENSE.txt', digest='X+6ZFkDOlnKesJCNt20yRg==', size=1594, local_path='C:\\Users\\Semal\\AppData\\Local\\wandb\\wandb\\artifacts\\staging\\tmpl8cxy4ew', skip_cache=False)

In [9]:
artifact.add_dir(path/'images', name='images')
artifact.add_dir(path/'labels', name='labels')

wandb: Adding directory to artifact (C:\Users\Semal\.fastai\data\bdd_simple_1k\images)... Done. 7.3s
wandb: Adding directory to artifact (C:\Users\Semal\.fastai\data\bdd_simple_1k\labels)... Done. 5.3s


In [10]:
image_files = get_image_files(path/'images', recurse=False)

In [11]:
table = _create_table(image_files, BDD_CLASSES)

In [12]:
artifact.add(table, "eda_table")

ArtifactManifestEntry(path='eda_table.table.json', digest='cjU2t62ro9oLsYNywMzsCQ==', size=610046, local_path='C:\\Users\\Semal\\AppData\\Local\\wandb\\wandb\\artifacts\\staging\\tmp8rtiduox', skip_cache=False)

In [13]:
run.log_artifact(artifact)

<Artifact bdd_simple_1k>

In [14]:
run.finish()

# Data Preparation

In [ ]:
import os, warnings
import pandas as pd
from sklearn.model_selection import StratifiedGroupKFold
warnings.filterwarnings('ignore')

In [61]:
run = wandb.init(project=WANDB_PROJECT, entity=ENTITY, job_type="data_split")

background_iou,▂▁▆▇█
bicycle_iou,▁▁▁▁▁
epoch,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_2,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_0,▁▁▂▂▂▄▄▄▆▆█████████████████▇▇▇▇▇▇▇▇▇▆▆▆▆
lr_1,▁▁▂▂▂▃▃▄▄▅▇▇█████████████▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆
lr_2,▁▁▂▂▃▅▅▅▆▆▇▇▇█████████████▇▇▇▇▇▇▇▇▆▆▆▆▆▆
miou,▂▁▆▇█
mom_0,████████▇▇▆▅▅▅▅▄▄▃▂▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃


In [62]:
raw_data_at = run.use_artifact(f'{RAW_DATA_AT}:latest')
path = Path(raw_data_at.download())

wandb: Downloading large artifact bdd_simple_1k:latest, 813.77MB. 4007 files... 
wandb:   4007 of 4007 files downloaded.  
Done. 0:0:20.7


In [63]:
path.ls()

(#5) [Path('C:/Users/Semal/Desktop/My Folder/Autonomous_vehicle_perception/artifacts/bdd_simple_1k-v0/eda_table.table.json'),Path('C:/Users/Semal/Desktop/My Folder/Autonomous_vehicle_perception/artifacts/bdd_simple_1k-v0/images'),Path('C:/Users/Semal/Desktop/My Folder/Autonomous_vehicle_perception/artifacts/bdd_simple_1k-v0/labels'),Path('C:/Users/Semal/Desktop/My Folder/Autonomous_vehicle_perception/artifacts/bdd_simple_1k-v0/LICENSE.txt'),Path('C:/Users/Semal/Desktop/My Folder/Autonomous_vehicle_perception/artifacts/bdd_simple_1k-v0/media')]

In [64]:
original_eda_table = raw_data_at.get("eda_table")

wandb: Downloading large artifact bdd_simple_1k:latest, 813.77MB. 4007 files... 
wandb:   4007 of 4007 files downloaded.  
Done. 0:0:15.7


In [65]:
fnames = original_eda_table.get_column("File_Name")
groups = [s.split('-')[0] for s in fnames]
y = original_eda_table.get_column("bicycle")

In [66]:
df = pd.DataFrame()
df['File_Name'] = fnames
df['fold'] = -1

In [67]:
cv = StratifiedGroupKFold(n_splits=10)
for i, (train_idxs, test_idxs) in enumerate(cv.split(fnames, y, groups)):
    df.loc[test_idxs, ['fold']] = i

In [68]:
df['Stage'] = 'train'
df.loc[df.fold == 0, ['Stage']] = 'test'
df.loc[df.fold == 1, ['Stage']] = 'valid'
del df['fold']
df.Stage.value_counts()

Stage
train    800
test     100
valid    100
Name: count, dtype: int64

In [69]:
df.to_csv('data_split.csv', index=False)

In [70]:
processed_data_at = wandb.Artifact(PROCESSED_DATA_AT, type="split_data")

In [71]:
processed_data_at.add_file('data_split.csv')
processed_data_at.add_dir(path)

wandb: Adding directory to artifact (C:\Users\Semal\Desktop\My Folder\Autonomous_vehicle_perception\artifacts\bdd_simple_1k-v0)... Done. 33.7s


In [72]:
data_split_table = wandb.Table(dataframe=df[["File_Name", "Stage"]])

In [73]:
join_table = wandb.JoinedTable(original_eda_table, data_split_table, "File_Name")

In [74]:
processed_data_at.add(join_table, "eda_table_data_split")

ArtifactManifestEntry(path='eda_table_data_split.joined-table.json', digest='RP3GwAqHdNFbNLo90M0bJw==', size=125, local_path='C:\\Users\\Semal\\AppData\\Local\\wandb\\wandb\\artifacts\\staging\\tmpqcxx5elf', skip_cache=False)

In [75]:
run.log_artifact(processed_data_at)
run.finish()

# Model Training

In [76]:
from fastai.callback.wandb import WandbCallback

In [77]:
from sklearn.metrics import ConfusionMatrixDisplay
from IPython.display import display, Markdown

CLASS_INDEX = {v:k for k,v in BDD_CLASSES.items()}

def iou_per_class(inp, targ):
    "Compute iou per class"
    iou_scores = []
    for c in range(inp.shape[0]):
        dec_preds = inp.argmax(dim=0)
        p = torch.where(dec_preds == c, 1, 0)
        t = torch.where(targ == c, 1, 0)
        c_inter = (p * t).float().sum().item()
        c_union = (p + t).float().sum().item()
        iou_scores.append(c_inter / (c_union - c_inter) if c_union > 0 else np.nan)
    return iou_scores

def create_row(sample, pred_label, prediction, class_labels):
    """"A simple function to create a row of (img, target, prediction, and scores...)"""
    (image, label) = sample
    # compute metrics
    iou_scores = iou_per_class(prediction, label)
    image = image.permute(1, 2, 0)
    row =[wandb.Image(
                image,
                masks={
                    "predictions": {
                        "mask_data": pred_label[0].numpy(),
                        "class_labels": class_labels,
                    },
                    "ground_truths": {
                        "mask_data": label.numpy(),
                        "class_labels": class_labels,
                    },
                },
            ),
            *iou_scores,
    ]
    return row

def create_iou_table(samples, outputs, predictions, class_labels):
    "Creates a wandb table with predictions and targets side by side"

    def _to_str(l):
        return [f'{str(x)} IoU' for x in l]
    
    items = list(zip(samples, outputs, predictions))
    
    table = wandb.Table(
        columns=["Image"]
        + _to_str(class_labels.values()),
    )
    # we create one row per sample
    for item in progress_bar(items):
        table.add_data(*create_row(*item, class_labels=class_labels))
    
    return table

def get_predictions(learner, test_dl=None, max_n=None):
    """Return the samples = (x,y) and outputs (model predictions decoded), and predictions (raw preds)"""
    test_dl = learner.dls.valid if test_dl is None else test_dl
    inputs, predictions, targets, outputs = learner.get_preds(
        dl=test_dl, with_input=True, with_decoded=True
    )
    x, y, samples, outputs = learner.dls.valid.show_results(
        tuplify(inputs) + tuplify(targets), outputs, show=False, max_n=max_n
    )
    return samples, outputs, predictions

    def value(self): return self.inter/(self.union-self.inter) if self.union > 0 else None

class MIOU(DiceMulti):
    @property
    def value(self):
        binary_iou_scores = np.array([])
        for c in self.inter:
            binary_iou_scores = np.append(binary_iou_scores, \
                                          self.inter[c]/(self.union[c]-self.inter[c]) if self.union[c] > 0 else np.nan)
        return np.nanmean(binary_iou_scores)
    
class IOU(DiceMulti):
    @property
    def value(self): 
        c=CLASS_INDEX[self.nm]
        return self.inter[c]/(self.union[c]-self.inter[c]) if self.union[c] > 0 else np.nan

class BackgroundIOU(IOU): nm = 'background'
class RoadIOU(IOU): nm = 'road'
class TrafficLightIOU(IOU): nm = 'traffic light'
class TrafficSignIOU(IOU): nm = 'traffic sign'
class PersonIOU(IOU): nm = 'person'
class VehicleIOU(IOU): nm = 'vehicle'
class BicycleIOU(IOU): nm = 'bicycle'


class IOUMacro(DiceMulti):
    @property
    def value(self): 
        c=CLASS_INDEX[self.nm]
        if c not in self.count: return np.nan
        else: return self.macro[c]/self.count[c] if self.count[c] > 0 else np.nan

    def reset(self): self.macro,self.count = {},{}

    def accumulate(self, learn):
        pred,targ = learn.pred.argmax(dim=self.axis), learn.y
        for c in range(learn.pred.shape[self.axis]):
            p = torch.where(pred == c, 1, 0)
            t = torch.where(targ == c, 1, 0)
            c_inter = (p*t).float().sum(dim=(1,2))
            c_union = (p+t).float().sum(dim=(1,2))
            m = c_inter / (c_union - c_inter)
            macro = m[~torch.any(m.isnan())]
            count = macro.shape[1]

            if count > 0:
                msum = macro.sum().item()
                if c in self.count:
                    self.count[c] += count
                    self.macro[c] += msum
                else:
                    self.count[c] = count
                    self.macro[c] = msum


class MIouMacro(IOUMacro):
    @property
    def value(self):
        binary_iou_scores = np.array([])
        for c in self.count:
            binary_iou_scores = np.append(binary_iou_scores, self.macro[c]/self.count[c] if self.count[c] > 0 else np.nan)
        return np.nanmean(binary_iou_scores)


class BackgroundIouMacro(IOUMacro): nm = 'background'
class RoadIouMacro(IOUMacro): nm = 'road'
class TrafficLightIouMacro(IOUMacro): nm = 'traffic light'
class TrafficSignIouMacro(IOUMacro): nm = 'traffic sign'
class PersonIouMacro(IOUMacro): nm = 'person'
class VehicleIouMacro(IOUMacro): nm = 'vehicle'
class BicycleIouMacro(IOUMacro): nm = 'bicycle'


def display_diagnostics(learner, dls=None, return_vals=False):
    """
    Display a confusion matrix for the unet learner.
    If `dls` is None it will get the validation set from the Learner
    
    You can create a test dataloader using the `test_dl()` method like so:
    >> dls = ... # You usually create this from the DataBlocks api, in this library it is get_data()
    >> tdls = dls.test_dl(test_dataframe, with_labels=True)
    
    See: https://docs.fast.ai/tutorial.pets.html#adding-a-test-dataloader-for-inference
    
    """
    probs, targs = learner.get_preds(dl = dls)
    preds = probs.argmax(dim=1)
    classes = list(BDD_CLASSES.values())
    y_true = targs.flatten().numpy()
    y_pred = preds.flatten().numpy()
    
    tdf, pdf = [pd.DataFrame(r).value_counts().to_frame(c) for r,c in zip((y_true, y_pred) , ['y_true', 'y_pred'])]
    countdf = tdf.join(pdf, how='outer').reset_index(drop=True).fillna(0).astype(int).rename(index=BDD_CLASSES)
    countdf = countdf/countdf.sum() 
    display(Markdown('### % Of Pixels In Each Class'))
    display(countdf.style.format('{:.1%}'))
    
    
    disp = ConfusionMatrixDisplay.from_predictions(y_true=y_true, y_pred=y_pred,
                                                   display_labels=classes,
                                                   normalize='pred')
    fig = disp.ax_.get_figure()
    fig.set_figwidth(10)
    fig.set_figheight(10) 
    disp.ax_.set_title('Confusion Matrix (by Pixels)', fontdict={'fontsize': 32, 'fontweight': 'medium'})
    fig.show()
    
    if return_vals: return countdf, disp

In [78]:
train_config = SimpleNamespace(
    framework="fastai",
    img_size=(180, 320),
    batch_size=8,
    augment=True, # use data augmentation
    epochs=10, 
    lr=2e-3,
    pretrained=True,  # whether to use pretrained encoder
    seed=42,
)

In [79]:
set_seed(train_config.seed, reproducible=True)

In [80]:
run = wandb.init(project=WANDB_PROJECT, entity=ENTITY, job_type="training", config=train_config)

In [81]:
processed_data_at = run.use_artifact(f'{PROCESSED_DATA_AT}:latest')
processed_dataset_dir = Path(processed_data_at.download())
df = pd.read_csv(processed_dataset_dir / 'data_split.csv')

wandb: Downloading large artifact bdd_simple_1k_split:latest, 813.25MB. 4010 files... 
wandb:   4010 of 4010 files downloaded.  
Done. 0:0:27.1


In [82]:
df = df[df.Stage != 'test'].reset_index(drop=True)
df['is_valid'] = df.Stage == 'valid'

In [83]:
def label_func(fname):
    return (fname.parent.parent/"labels")/f"{fname.stem}_mask.png"

In [84]:
df["image_fname"] = [processed_dataset_dir/f'images/{f}' for f in df.File_Name.values]
df["label_fname"] = [label_func(f) for f in df.image_fname.values]

In [85]:
def get_data(df, bs=4, img_size=(180, 320), augment=True):
    block = DataBlock(blocks=(ImageBlock, MaskBlock(codes=BDD_CLASSES)),
                  get_x=ColReader("image_fname", suff='.jpg'),
                  get_y=ColReader("label_fname"),
                  splitter=ColSplitter(),
                  item_tfms=Resize(img_size),
                  batch_tfms=aug_transforms() if augment else None,
                 )
    return block.dataloaders(df, bs=bs)

In [86]:
config = wandb.config

In [87]:
dls = get_data(df, bs=config.batch_size, img_size=config.img_size, augment=config.augment)

In [88]:
print(df[['image_fname']].head())

                                                                                                                image_fname
0  C:\Users\Semal\Desktop\My Folder\Autonomous_vehicle_perception\artifacts\bdd_simple_1k_split-v2\images\0027eed2-09c90000
1  C:\Users\Semal\Desktop\My Folder\Autonomous_vehicle_perception\artifacts\bdd_simple_1k_split-v2\images\0027eed2-09c90001
2  C:\Users\Semal\Desktop\My Folder\Autonomous_vehicle_perception\artifacts\bdd_simple_1k_split-v2\images\00aad4a0-ee8135fe
3  C:\Users\Semal\Desktop\My Folder\Autonomous_vehicle_perception\artifacts\bdd_simple_1k_split-v2\images\00d79c0a-23befe54
4  C:\Users\Semal\Desktop\My Folder\Autonomous_vehicle_perception\artifacts\bdd_simple_1k_split-v2\images\00e69ee0-9656df95


In [89]:
metrics = [MIOU(), BackgroundIOU(), RoadIOU(), TrafficLightIOU(), \
           TrafficSignIOU(), PersonIOU(), VehicleIOU(), BicycleIOU()]

learn = unet_learner(dls, arch=resnet18, pretrained=config.pretrained, metrics=metrics)

In [90]:
callbacks = [
    SaveModelCallback(monitor='miou'),
    WandbCallback(log_preds=False, log_model=True)
]

In [91]:
learn.fit_one_cycle(config.epochs, config.lr, cbs=callbacks)

epoch,train_loss,valid_loss,miou,background_iou,road_iou,traffic_light_iou,traffic_sign_iou,person_iou,vehicle_iou,bicycle_iou,time
0,0.498824,0.359526,0.308615,0.863343,0.719259,0.000000,0.000000,0.000000,0.577702,0.000000,2:05:19
1,0.378712,0.397279,0.300791,0.858751,0.727972,0.000000,0.000000,0.000000,0.518813,0.000000,59:41
2,0.342693,0.338868,0.331844,0.891564,0.797909,0.000000,0.000000,0.000000,0.633436,0.000000,1:25:11
3,0.318976,0.273293,0.342093,0.901053,0.800810,0.000000,0.000000,0.000000,0.692787,0.000000,1:17:44
4,0.275806,0.247638,0.348584,0.904488,0.816115,0.000000,0.000000,0.000000,0.719485,0.000000,1:02:50
5,0.243826,0.235586,0.360453,0.919278,0.841591,0.020700,0.000000,0.000000,0.741602,0.000000,1:20:17
6,0.228935,0.238623,0.360150,0.916151,0.837224,0.032097,0.000000,0.000000,0.735578,0.000000,1:03:55
7,0.205480,0.226565,0.370939,0.920979,0.848697,0.069279,0.000000,0.000000,0.757620,0.000000,1:00:20
8,0.190401,0.230159,0.379614,0.922503,0.846163,0.131652,0.000000,0.000000,0.756982,0.000000,1:02:19
9,0.179647,0.227563,0.381523,0.922796,0.849223,0.131913,0.000000,0.000000,0.766726,0.000000,1:04:37


Better model found at epoch 0 with miou value: 0.3086148177722196.
Better model found at epoch 2 with miou value: 0.3318441515519253.
Better model found at epoch 3 with miou value: 0.34209279016571875.
Better model found at epoch 4 with miou value: 0.3485840421328832.
Better model found at epoch 5 with miou value: 0.360452994157827.
Better model found at epoch 7 with miou value: 0.3709392022500198.
Better model found at epoch 8 with miou value: 0.379614366514122.
Better model found at epoch 9 with miou value: 0.38152260904552954.


In [93]:
samples, outputs, predictions = get_predictions(learn)
table = create_iou_table(samples, outputs, predictions, BDD_CLASSES)
wandb.log({"pred_table":table})

In [94]:
scores = learn.validate()
metric_names = ['final_loss'] + [f'final_{x.name}' for x in metrics]
final_results = {metric_names[i] : scores[i] for i in range(len(scores))}
for k,v in final_results.items(): 
    wandb.summary[k] = v

In [95]:
wandb.finish()

background_iou,▂▁▅▆▆█▇███
bicycle_iou,▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_2,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_0,▁▁▃▅▆█████████████▇▇▇▆▆▆▅▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁
lr_1,▁▁▁▂▃▃▃▅▆▆████████▇▇▆▆▅▅▅▄▄▄▄▃▃▂▂▂▂▂▁▁▁▁
lr_2,▄▄▅▆▇████▇▇▇▇▇▇▆▅▅▅▅▄▄▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
miou,▂▁▄▅▅▆▆▇██
mom_0,█▆▆▅▅▄▃▃▃▁▁▁▁▁▂▂▃▃▃▄▄▄▅▆▆▆▆▆▇▇▇▇████████
